## This script we double check whether our redshift errors are added correctly

In [7]:
import sys
import fitsio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from astropy.table import Table,join,Column
from cosmoprimo.fiducial import DESI, AbacusSummit

from scipy.interpolate import interp1d

sys.path.append('/global/homes/s/shengyu/desi_y3_redshift_errors/main/')
from helper import REDSHIFT_BIN_OVERALL, REDSHIFT_ABACUSHF_v1
from helper import CSPEED, TRACER_CUTSKY_INFO
from helper import GET_REPEATS_DV, GET_CTHR, GET_REPEATS_NUMBER
from cat_tools import get_positions_weights

ABACUSHF_DIR = f'/global/cfs/projectdirs/desi/mocks/cai/abacus_HF/DR2_v1.0'
LOCAL_DIR = '/pscratch/sd/s/shengyu/galaxies/catalogs/Y3/AbacusHF-v1'
REPEAT_DIR = '/pscratch/sd/s/shengyu/repeats/DA2/loa-v1'

def zfmt(x):
    return f"{x:.3f}".replace(".", "p")

BOXSIZE = 2000
BASE_DIR = '/pscratch/sd/s/shengyu/galaxies/catalogs/Y3/AbacusHF-v1'

## For cubic box mocks 

But we add the periodic condition, so hard to say if pdf(Z_OBS-Z_RSD) ~ pdf(dv)

In [5]:
tracers = ['LRG','ELG','QSO']
mockids = range(25)

for tracer in tracers:
    lenf = len(REDSHIFT_ABACUSHF_v1[tracer])
    (zmin, zmax) = REDSHIFT_BIN_OVERALL[tracer]
    fig = plt.figure(figsize = (lenf*5, 4))
    gs = gridspec.GridSpec(1, lenf, wspace=0.2, hspace=0.25)
    for indz, z in enumerate(REDSHIFT_ABACUSHF_v1[tracer]):
        ax = fig.add_subplot(gs[indz])
        for mock_id in mockids:
            mock_id03 =  f"{mock_id:03}"
            cubic_name = f'/abacus_HF_{tracer}_{zfmt(z)}_DR2_v1.0_AbacusSummit_base_c000_ph{mock_id03}_clustering.dat.fits'
            data_fn = BASE_DIR+ f'/Boxes/{tracer}/z{z:.3f}/AbacusSummit_base_c000_ph{mock_id03}'+cubic_name
            data = Table.read(data_fn)

        dv = GET_REPEATS_DV
        plt.hist(dv, density =True)

/pscratch/sd/s/shengyu/galaxies/catalogs/Y3/AbacusHF-v1/Boxes/LRG/z0.500/AbacusSummit_base_c000_ph000/abacus_HF_LRG_0p500_DR2_v1.0_AbacusSummit_base_c000_ph000_clustering.dat.fits
/pscratch/sd/s/shengyu/galaxies/catalogs/Y3/AbacusHF-v1/Boxes/LRG/z0.500/AbacusSummit_base_c000_ph001/abacus_HF_LRG_0p500_DR2_v1.0_AbacusSummit_base_c000_ph001_clustering.dat.fits
/pscratch/sd/s/shengyu/galaxies/catalogs/Y3/AbacusHF-v1/Boxes/LRG/z0.500/AbacusSummit_base_c000_ph002/abacus_HF_LRG_0p500_DR2_v1.0_AbacusSummit_base_c000_ph002_clustering.dat.fits
/pscratch/sd/s/shengyu/galaxies/catalogs/Y3/AbacusHF-v1/Boxes/LRG/z0.500/AbacusSummit_base_c000_ph003/abacus_HF_LRG_0p500_DR2_v1.0_AbacusSummit_base_c000_ph003_clustering.dat.fits
/pscratch/sd/s/shengyu/galaxies/catalogs/Y3/AbacusHF-v1/Boxes/LRG/z0.500/AbacusSummit_base_c000_ph004/abacus_HF_LRG_0p500_DR2_v1.0_AbacusSummit_base_c000_ph004_clustering.dat.fits
/pscratch/sd/s/shengyu/galaxies/catalogs/Y3/AbacusHF-v1/Boxes/LRG/z0.500/AbacusSummit_base_c000_ph00

## For the Cutsky mocks

In [ ]:
# load the data
tracers = ['LRG','ELG','QSO']
mock_id = '000'

for i, tracer in enumerate(tracers):
    lenf = len(REDSHIFT_ABACUSHF_v1[tracer])
    tracer_type = TRACER_CUTSKY_INFO[tracer]['tracer_type']
    fit_range   = TRACER_CUTSKY_INFO[tracer]['fit_range']
    (zmin, zmax) = REDSHIFT_BIN_OVERALL[tracer]
    fig = plt.figure(figsize = (lenf*5, 4))
    gs = gridspec.GridSpec(1, lenf, wspace=0.2, hspace=0.25)
    for indz, z in enumerate(REDSHIFT_ABACUSHF_v1[tracer]):
        ax = fig.add_subplot(gs[indz]) 
        cutsky_name = f'cutsky_abacusHF_DR2_{tracer_type}_z{zfmt(z)}_zcut_{fit_range}_clustering.dat.fits'
        data_fn = LOCAL_DIR+ f'/Cutsky/{tracer_type[:3]}/z{z:.3f}/AbacusSummit_base_c000_ph{mock_id}/forclustering/'+cutsky_name
        data = Table.read(data_fn)
        bins = np.arange(zmin, zmax, 0.01)
        for Zcol in ['Z', 'Z_OBS_GLOBAL', 'Z_OBS_BIN']:
            plt.hist(data[Zcol],  bins=bins, alpha=0.6, histtype='step', label = f'{tracer} {Zcol}')
        ax.legend(loc=3, fontsize=10)
        ax.grid(True, alpha=0.3)